In [5]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

In [6]:
#Hyperparameters
learning_rate = 1e-3 #how much to update models parameters at each batch/epoch. Smaller values yield slow learning speed, while large values may result in unpredictable behavior during training.
batch_size = 64 #the number of data samples propagated through the network before the parameters are updated
epochs = 5 #Number of Epochs - the number times to iterate over the dataset

In [7]:
#Optimization Loop
#Once we set our hyperparameters, we can then train and optimize our model with an optimization loop.
# Each iteration of the optimization loop is called an epoch.

# Each epoch consists of two main parts:
# The Train Loop - iterate over the training dataset and try to converge to optimal parameters.

# The Validation/Test Loop - iterate over the test dataset to check if model performance is improving.

In [8]:
#initialize the loss function
loss_fn = nn.CrossEntropyLoss()

In [9]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [10]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    # Set the model to training mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * batch_size + len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    # Set the model to evaluation mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    # Evaluating the model with torch.no_grad() ensures that no gradients are computed during test mode
    # also serves to reduce unnecessary gradient computations and memory usage for tensors with requires_grad=True
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [11]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.304344  [   64/60000]
loss: 2.285619  [ 6464/60000]
loss: 2.255595  [12864/60000]
loss: 2.248112  [19264/60000]
loss: 2.243827  [25664/60000]
loss: 2.207092  [32064/60000]
loss: 2.222519  [38464/60000]
loss: 2.185263  [44864/60000]
loss: 2.176424  [51264/60000]
loss: 2.140751  [57664/60000]
Test Error: 
 Accuracy: 39.5%, Avg loss: 2.135951 

Epoch 2
-------------------------------
loss: 2.148057  [   64/60000]
loss: 2.137710  [ 6464/60000]
loss: 2.063198  [12864/60000]
loss: 2.088434  [19264/60000]
loss: 2.047014  [25664/60000]
loss: 1.968406  [32064/60000]
loss: 2.016998  [38464/60000]
loss: 1.921384  [44864/60000]
loss: 1.926417  [51264/60000]
loss: 1.856470  [57664/60000]
Test Error: 
 Accuracy: 53.1%, Avg loss: 1.849664 

Epoch 3
-------------------------------
loss: 1.880380  [   64/60000]
loss: 1.853364  [ 6464/60000]
loss: 1.718309  [12864/60000]
loss: 1.783139  [19264/60000]
loss: 1.678867  [25664/60000]
loss: 1.615129  [32064/600

In [12]:
import torch
import torchvision.models as models

In [13]:
model = models.vgg16(weights='IMAGENET1K_V1')
torch.save(model.state_dict(), 'model_weights.pth')

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to C:\Users\mibenama/.cache\torch\hub\checkpoints\vgg16-397923af.pth
100.0%


In [14]:
model = models.vgg16() # we do not specify ``weights``, i.e. create untrained model
model.load_state_dict(torch.load('model_weights.pth', weights_only=True))
model.eval()

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [15]:
torch.save(model, 'model.pth')

In [17]:
model = torch.load('model.pth', weights_only=False)